In [ ]:
%pip install -e ..
%restart_python

In [ ]:
from github import Github
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
# ------------------------------------------------------------------------------
# Databricks Context
# ------------------------------------------------------------------------------
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

In [ ]:
# --- Configuration ---
GITHUB_TOKEN = dbutils.secrets.get(
    scope="git_token",
    key="pat_token"
)
REPO_NAME = "sarath-knackforge/honeywell-cicd"
BASE_BRANCH = "main"          # The branch you want to merge changes INTO
HEAD_BRANCH = "feature/stage" # The branch you want to merge changes FROM
PR_TITLE = "Automated PR from Python Script"
PR_BODY = "This pull request was created automatically using the PyGithub library."
# ---------------------

def create_pull_request(token, repo_name, base_branch, head_branch, title, body):
    """
    Creates a pull request in a GitHub repository.
    """
    try:
        # Authenticate with GitHub
        g = Github(token)
        repo = g.get_repo(repo_name)
        
        # Check if the head branch has changes compared to the base branch
        # The head parameter is the source of the changes, and base is the target.
        comparison = repo.compare(base_branch, head_branch)
        if comparison.total_commits == 0:
            print(f"No changes found between '{base_branch}' and '{head_branch}'. PR not needed.")
            return

        # Create the pull request
        pr = repo.create_pull(
            title=title,
            body=body,
            base=base_branch,
            head=head_branch
        )
        print(f"Successfully created pull request: [PR #{pr.number}]({pr.html_url})")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    create_pull_request(
        GITHUB_TOKEN,
        REPO_NAME,
        BASE_BRANCH,
        HEAD_BRANCH,
        PR_TITLE,
        PR_BODY
    )